In [6]:
import os
import zipfile

# Path to the downloaded dataset
path_to_dataset = "/content/kdkd.zip"

# Since path_to_dataset is a zip file, it doesn't contain other zip files within it
# We'll directly extract this zip file
extract_to = os.path.join(os.path.dirname(path_to_dataset), "extracted")

# Create a directory for extraction if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(path_to_dataset, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Extracted {path_to_dataset} to {extract_to}")

Extracted /content/kdkd.zip to /content/extracted


In [7]:
import os
print(os.path.exists("/content/kdkd.zip"))


True


In [8]:
import os
import random
import numpy as np
from PIL import Image, ImageOps, ImageEnhance
import cv2


In [93]:
## Augmentasi dulu

In [9]:
def flip_horizontal(image):
    if random.random() > 0.5:  # 50% probability
        return ImageOps.mirror(image)
    return image


In [10]:
def rotate(image):
    if random.random() > 0.5:  # 50% probability
        angle = random.uniform(-30, 30)  # Rotate within +/- 30 degrees
        return image.rotate(angle)
    return image


In [11]:
def scale(image):
    scale_factor = random.uniform(0.8, 1.2)  # Scale up/down by 20%
    w, h = image.size
    new_size = (int(w * scale_factor), int(h * scale_factor))
    # Use Image.Resampling.LANCZOS instead of Image.ANTIALIAS
    image = image.resize(new_size, Image.Resampling.LANCZOS)
    return image.crop((0, 0, w, h))  # Crop back to original size


In [12]:
def translate(image):
    max_translate = 0.1  # 10% of the image size
    w, h = image.size
    x_translate = int(random.uniform(-max_translate, max_translate) * w)
    y_translate = int(random.uniform(-max_translate, max_translate) * h)
    return Image.fromarray(np.roll(np.roll(np.array(image), x_translate, axis=1), y_translate, axis=0))


In [13]:
def blur(image):
    if random.random() > 0.8:  # 20% probability
        img_array = np.array(image)
        blur_strength = random.randint(1, 3)  # Max blur strength: 3 pixels
        img_array = cv2.GaussianBlur(img_array, (2 * blur_strength + 1, 2 * blur_strength + 1), 0)
        return Image.fromarray(img_array)
    return image


In [14]:
def resize(image):
    # Use Image.Resampling.LANCZOS instead of Image.ANTIALIAS
    return image.resize((100, 100), Image.Resampling.LANCZOS)

In [15]:
def add_gaussian_noise(image):
    img_array = np.array(image)
    mean = 0
    sigma = random.uniform(0.01, 0.05) * 255
    noise = np.random.normal(mean, sigma, img_array.shape).astype('uint8')
    noisy_img = cv2.add(img_array, noise)
    return Image.fromarray(np.clip(noisy_img, 0, 255))


In [16]:
def adjust_brightness(image):
    enhancer = ImageEnhance.Brightness(image)
    factor = random.uniform(0.8, 1.2)  # Random brightness factor
    return enhancer.enhance(factor)

In [17]:
def adjust_contrast(image):
    enhancer = ImageEnhance.Contrast(image)
    factor = random.uniform(0.8, 1.2)  # Random contrast factor
    return enhancer.enhance(factor)

In [18]:
def adjust_sharpness(image):
    enhancer = ImageEnhance.Sharpness(image)
    factor = random.uniform(0.8, 2.0)  # Random sharpness factor
    return enhancer.enhance(factor)

In [19]:
def adjust_color(image):
    enhancer = ImageEnhance.Color(image)
    factor = random.uniform(0.8, 1.5)  # Random color factor
    return enhancer.enhance(factor)

In [20]:
def crop_image(image):
    w, h = image.size
    crop_factor = random.uniform(0.9, 1.0)  # Crop up to 10% of the image
    left = random.uniform(0, w * (1 - crop_factor))
    top = random.uniform(0, h * (1 - crop_factor))
    right = left + crop_factor * w
    bottom = top + crop_factor * h
    return image.crop((left, top, right, bottom)).resize((w, h), Image.Resampling.LANCZOS)


In [35]:
import os
import shutil
from PIL import Image

# Path to the dataset
original_dataset_path = "/content/extracted/Citra BISINDO"
augment_dataset_path = "/content/augmented/Citra BISINDO"

# Create the augmentasi directory if it doesn't exist
os.makedirs(augment_dataset_path, exist_ok=True)

# List of augmentation functions
augmentation_functions = [
    flip_horizontal,
    rotate,
    scale,
    translate,
    blur,
    resize,
    add_gaussian_noise,
    adjust_brightness,
    adjust_contrast,
    adjust_sharpness,
    adjust_color,
    crop_image
]

# Iterate through each folder (A-Z)
for folder in sorted(os.listdir(original_dataset_path)):
    folder_path = os.path.join(original_dataset_path, folder)
    if os.path.isdir(folder_path):
        augment_folder_path = os.path.join(augment_dataset_path, folder)
        os.makedirs(augment_folder_path, exist_ok=True)

        # Iterate through each file in the folder
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            if file_name.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff')):
                try:
                    # Open the image
                    original_image = Image.open(file_path).convert("RGB")

                    # Save the original image to the augmented folder
                    original_image.save(os.path.join(augment_folder_path, file_name))

                    # Apply each augmentation function once
                    for func in augmentation_functions:
                        augmented_image = func(original_image)

                        # Save the augmented image
                        new_file_name = f"{os.path.splitext(file_name)[0]}_{func.__name__}{os.path.splitext(file_name)[1]}"
                        augmented_image.save(os.path.join(augment_folder_path, new_file_name))
                except Exception as e:
                    print(f"Error processing file {file_name}: {e}")

print("Augmentation completed and saved in:", augment_dataset_path)


Augmentation completed and saved in: /content/augmented/Citra BISINDO


In [94]:


import os

def count_images_per_folder(root_dir):
  """Counts the number of images in each subfolder of the root directory.

  Args:
    root_dir: The path to the root directory.

  Returns:
    A dictionary where keys are folder names and values are the number of images
    in each folder.
  """

  image_counts = {}
  for folder_name in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder_name)
    if os.path.isdir(folder_path):
      image_count = 0
      for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
          image_count += 1
      image_counts[folder_name] = image_count
  return image_counts

# Example usage:
image_counts = count_images_per_folder("/content/augmented/Citra BISINDO")
for folder, count in image_counts.items():
  print(f"Folder: {folder}, Image Count: {count}")

Folder: X, Image Count: 156
Folder: D, Image Count: 156
Folder: Y, Image Count: 156
Folder: Q, Image Count: 156
Folder: K, Image Count: 156
Folder: U, Image Count: 156
Folder: R, Image Count: 156
Folder: S, Image Count: 156
Folder: P, Image Count: 156
Folder: E, Image Count: 156
Folder: I, Image Count: 156
Folder: T, Image Count: 156
Folder: L, Image Count: 156
Folder: A, Image Count: 156
Folder: M, Image Count: 156
Folder: O, Image Count: 156
Folder: F, Image Count: 156
Folder: W, Image Count: 156
Folder: G, Image Count: 156
Folder: C, Image Count: 156
Folder: B, Image Count: 156
Folder: H, Image Count: 156
Folder: V, Image Count: 156
Folder: J, Image Count: 156
Folder: Z, Image Count: 156
Folder: N, Image Count: 156


In [95]:


import shutil
import os

def zip_folder(folder_path, zip_file_name):
    """Zips the specified folder.

    Args:
        folder_path: The path to the folder to zip.
        zip_file_name: The name of the zip file to create.
    """
    try:
        shutil.make_archive(zip_file_name, 'zip', folder_path)
        print(f"Successfully zipped '{folder_path}' to '{zip_file_name}.zip'")
    except FileNotFoundError:
        print(f"Error: Folder '{folder_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
folder_to_zip = "/content/augmented/Citra BISINDO"
zip_file_name = "/content/Citra_BISIND(Augmented)"  # Specify the desired output zip file name without extension

zip_folder(folder_to_zip, zip_file_name)

Successfully zipped '/content/augmented/Citra BISINDO' to '/content/Citra_BISIND(Augmented).zip'
